In [16]:
import numpy as np

In [26]:
#Define Main Classes
class newAnkleMechanismKinematics:
    def __init__(self, base_mount = np.array([0,0,0]), rod_mount = np.array([0,0,0])):
        self.base_mount = base_mount
        self.rod_mount = rod_mount
        

class actuatorParams:
    def __init__(self,zeroLength, strokeLength, safteyOffset):
        self.zeroLength = zeroLength
        self.strokeLength = strokeLength
        self.safteyOffset = safteyOffset
        self.minLength = self.zeroLength + self.safteyOffset
        self.maxLength = self.zeroLength + self.strokeLength - self.safteyOffset

class jointParams:
    def __init__(self, axisOffset = np.array([0,0,0])):
        self.axisOffset = axisOffset



In [25]:
#Define Main Parameters

leftOutsideAttachement = newAnkleMechanismKinematics([0.01, 0.018, -0.0306], [0.0388118, 0.018, 0.0178769])
leftInsideAttachement = newAnkleMechanismKinematics([0.01, -0.018, -0.306], [0.0388118, -0.018, 0.017869])

newAnkleActuator = actuatorParams(0.278066, 0.070, 0.004)

RollPitchAxisOffset = jointParams([0, 0, 0.0081])

In [17]:

def generateRomPlots(mechanismKinematics, actuatorParams, n, title, color):
    thetaPitchVec = np.linspace(-80,80, n)
    thetaRollVec = np.linspace(-25,25, n)

    outsideActuatorLengthsMatrix = np.zeros((10,10))
    insideActuatorLengthsMatrix = np.zeros((10,10))

    for i in range(len(thetaPitchVec)):
        for j in range(len(thetaRollVec)):
            thetaPitch = thetaPitchVec[i]
            thetaRoll = thetaRollVec[j]
            #actuatorLengths = 
            print(thetaPitch)
            print(thetaRoll)

In [18]:
generateRomPlots(1, 1, 10, "name", "color")

-80.0
-25.0
-80.0
-19.444444444444443
-80.0
-13.88888888888889
-80.0
-8.333333333333336
-80.0
-2.7777777777777786
-80.0
2.7777777777777786
-80.0
8.333333333333329
-80.0
13.888888888888886
-80.0
19.444444444444443
-80.0
25.0
-62.22222222222222
-25.0
-62.22222222222222
-19.444444444444443
-62.22222222222222
-13.88888888888889
-62.22222222222222
-8.333333333333336
-62.22222222222222
-2.7777777777777786
-62.22222222222222
2.7777777777777786
-62.22222222222222
8.333333333333329
-62.22222222222222
13.888888888888886
-62.22222222222222
19.444444444444443
-62.22222222222222
25.0
-44.44444444444444
-25.0
-44.44444444444444
-19.444444444444443
-44.44444444444444
-13.88888888888889
-44.44444444444444
-8.333333333333336
-44.44444444444444
-2.7777777777777786
-44.44444444444444
2.7777777777777786
-44.44444444444444
8.333333333333329
-44.44444444444444
13.888888888888886
-44.44444444444444
19.444444444444443
-44.44444444444444
25.0
-26.666666666666664
-25.0
-26.666666666666664
-19.444444444444443
-2

In [19]:
#Rodrigues Rotation
def rodrigues_rot(v, k, theta):
    #print(v)
    [m,n] = v.shape
    #print(m)
    #print(n)
    
    if ((m!=3) and (n!=3)):
        print("Error: Input vector is not three dimensional")
    elif ((v.size) != (k.size)):
        #print(v.shape)
        #print(k.shape)
        print("Error: Rotation vector v and axis k have different dimensions")                                                                                      
    
    k = k/math.sqrt(k[0]**2 + k[1]**2 + k[2]**2) #Normalize roation axis
    #print(k)

    No = v.size / 3 #number of vectors in array

    v_rot = np.empty((m,n))
    #v_rot = v #initialize rotated vector array
    #print(v_rot)

    if (n == 3):
        crosskv = v[0,] #initialize cross product k and v with right dim
        #print(crosskv)
        for i in range (int(No)):
            v_row = (v[i,])
            crosskv[0] = k[1] * v[i,2] - k[2] * v[i,1] 
            crosskv[1] = k[2] * v[i,0] - k[0] * v[i,2] 
            crosskv[2] = k[0] * v[i,1] - k[1] * v[i,0]
            v_rot[i,] = np.cos(theta) * v_row + (crosskv) * np.sin(theta) + k * (np.dot(k,v_row)) * (1 - np.cos(theta))
    
    else: #If m == 3 and n != 3
        crosskv = v[:,0] #Initalize cross product k and v with right dim
        for i in range (int(No)):
            crosskv[0] = k[1] * v[2,i] - k[2] * v[1,i] 
            crosskv[1] = k[2] * v[0,i] - k[0] * v[2,i] 
            crosskv[2] = k[0] * v[1,i] - k[1] * v[0,i]
            v_rot[:,i] = np.cos(theta) * v[:,i] + (crosskv) * np.sin(theta) + k * (np.dot(k,(v[:,i]))) * (1 - np.cos(theta)) 
    return v_rot   


In [20]:
def computeActuatorLengthsExtended(mechanismKinematics, thetaPitch, thetaRoll):
    x = mechanismKinematics.leftOutsideAnkleFootAttachment[0]
    y_outside = mechanismKinematics.leftOutsideAnkleFootAttachment[1]
    y_inside = mechanismKinematics.leftInsideAnkleFootAttachment[1]
    z = mechanismKinematics.leftOutsideAnkleFootAttachment[2]

    #put angles in same direction as onshape
    thetaPitch = -thetaPitch

    axisOffset = mechanismKinematics.RollPitchAxisOffeset
    pitchAxis = np.add(axisOffset, np.array([x, 0, 0]))  
    rollAxis = np.array([x, 0, z])
    insideActuator = np.array([x, y_outside, z])
    outsideAcuator = np.array([x, y_inside, z])

    #Rotate axisOffset & pitchAxis around y-axis by thetaPitch
    v = np.array([axisOffset, pitchAxis, rollAxis, insideActuator, outsideAcuator])

    #Y-Axis Unit Vector
    yAxis = np.array([0, 1, 0])

    v = rodrigues_rot(v, k, thetaPitch)
    print(v)

    